In [19]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
import gymnasium as gym
import pygame
import random
from collections import deque

import time
from pathlib import Path
import datetime
import tempfile
import json
import shutil
import imageio

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#Override device to cpu
#device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [21]:
#env_name = 'MountainCar-v0'
env_name = 'CartPole-v1'

env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dims = [env.action_space.n]
#action_dims = env.action_space.n


In [22]:
print(f"State dim: {state_dim}")
print(f"Action dim: {action_dims}")

State dim: 4
Action dim: [2]


In [23]:
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer


class Agent(nn.Module):
    def __init__(self, state_size, action_sizes):
        super().__init__()
        self.action_sizes = action_sizes
        self.state_size = state_size

        self.critic = nn.Sequential(
            layer_init(nn.Linear(state_size, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 1), std=1.0),
        )
        self.actor = nn.Sequential(
            layer_init(nn.Linear(state_size, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, action_sizes), std=0.01),
        )

    def get_value(self, x):
        return self.critic(x)

    def get_action_and_value(self, x, action=None):
        logits = self.actor(x)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(x)

In [24]:
# Create the agent
agent = Agent(state_dim, action_dims[0]).to(device)

In [25]:
# Test the agent by getting a batch from the environment
state,_ = env.reset()
state = torch.tensor(state, dtype=torch.float32).to(device)
action, log_prob, entropy, value = agent.get_action_and_value(state)
print(f"State: {state}")
print(f"Action: {action.item()}")
print(f"Log prob: {log_prob.item()}")
print(f"Entropy: {entropy.item()}")
print(f"Value: {value.item()}")


State: tensor([ 0.0158, -0.0476, -0.0371, -0.0185], device='cuda:0')
Action: 1
Log prob: -0.6929978132247925
Entropy: 0.6931471824645996
Value: 0.011103712022304535


In [26]:
# Test the agent by getting several observations and actions
state,_ = env.reset()
state = [state, state, state]
actions = [0,1,0]
state = torch.tensor(state, dtype=torch.float32).to(device)
actions = torch.tensor(actions, dtype=torch.int64).to(device)
_, log_prob, entropy, value = agent.get_action_and_value(state, actions)
print(f"State: {state}")
print(f"Actions: {actions}")
print(f"Log prob: {log_prob}")
print(f"Entropy: {entropy}")
print(f"Value: {value}")



State: tensor([[ 0.0499, -0.0310, -0.0288,  0.0362],
        [ 0.0499, -0.0310, -0.0288,  0.0362],
        [ 0.0499, -0.0310, -0.0288,  0.0362]], device='cuda:0')
Actions: tensor([0, 1, 0], device='cuda:0')
Log prob: tensor([-0.6934, -0.6929, -0.6934], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
Entropy: tensor([0.6931, 0.6931, 0.6931], device='cuda:0', grad_fn=<NegBackward0>)
Value: tensor([[0.0135],
        [0.0135],
        [0.0135]], device='cuda:0', grad_fn=<AddmmBackward0>)


# Setup environment variables

In [27]:

learning_rate = 2.5e-4
num_steps = 200
num_envs = 1
total_timesteps = 500000
num_minibatches = 4
update_epochs = 4
batch_size = num_steps * num_envs
minibatch_size = int(batch_size // num_minibatches)
anneal_lr = True
target_kl = None



gae = True #To fix later
norm_adv = True
clip_vloss = True #To fix later
gamma = 0.99
gae_lambda = 0.95
update_epochs = 3
clip_coef = 0.2
ent_coef = 0.01
vf_coef = 0.5
max_grad_norm = 0.5



In [28]:
type((len(action_dims),))


tuple

In [29]:
# ALGO Logic: Storage setup
obs = torch.zeros((num_steps, num_envs, state_dim)).to(device) #Need to fix dims later
actions = torch.zeros((num_steps, num_envs) + (len(action_dims),)).to(device) #Need to fix dims later
logprobs = torch.zeros((num_steps, num_envs)).to(device)
rewards = torch.zeros((num_steps, num_envs)).to(device)
dones = torch.zeros((num_steps, num_envs)).to(device)
values = torch.zeros((num_steps, num_envs)).to(device)

In [30]:
actions.shape

torch.Size([200, 1, 1])

In [31]:
optimizer = optim.Adam(agent.parameters(), lr=learning_rate, eps=1e-5)

In [32]:
global_step = 0
start_time = time.time()

num_updates = total_timesteps // batch_size

for update in range(1, num_updates + 1):
        # Annealing the rate if instructed to do so.
        if anneal_lr:
            frac = 1.0 - (update - 1.0) / num_updates
            lrnow = frac * learning_rate
            optimizer.param_groups[0]["lr"] = lrnow

        #Reset the environment
        x,_ = env.reset()
        next_obs = torch.Tensor(x).to(device)
        next_done = torch.zeros(num_envs).to(device)

        for step in range(0, num_steps):
            global_step += 1 * num_envs
            obs[step] = next_obs
            dones[step] = next_done

            # ALGO LOGIC: action logic
            with torch.no_grad():
                action, logprob, _, value = agent.get_action_and_value(next_obs)
                values[step] = value.flatten()
            actions[step] = action
            logprobs[step] = logprob


            next_obs, reward, done, info, _ = env.step(action.cpu().numpy())

            # print(f'State: {next_obs}')
            # print(f'action: {action}')
            # print(f'Reward: {reward}')
            # print(f'Done: {done}')
            



            rewards[step] = torch.tensor(reward).to(device).view(-1)
            
            next_obs = torch.Tensor(next_obs).to(device)
            next_done = torch.Tensor([int(done)]).to(device)

            #Not sure if this is needed...
            # if done:
            #     break

        


        # bootstrap value if not done
        with torch.no_grad():
            next_value = agent.get_value(next_obs).reshape(1, -1)
            if gae:
                advantages = torch.zeros_like(rewards).to(device)
                lastgaelam = 0
                for t in reversed(range(num_steps)):
                    if t == num_steps - 1:
                        nextnonterminal = 1.0 - next_done
                        nextvalues = next_value
                    else:
                        nextnonterminal = 1.0 - dones[t + 1]
                        nextvalues = values[t + 1]
                    delta = rewards[t] + gamma * nextvalues * nextnonterminal - values[t]
                    advantages[t] = lastgaelam = delta + gamma * gae_lambda * nextnonterminal * lastgaelam
                returns = advantages + values
            else:
                returns = torch.zeros_like(rewards).to(device)
                for t in reversed(range(num_steps)):
                    if t == num_steps - 1:
                        nextnonterminal = 1.0 - next_done
                        next_return = next_value
                    else:
                        nextnonterminal = 1.0 - dones[t + 1]
                        next_return = returns[t + 1]
                    returns[t] = rewards[t] + gamma * nextnonterminal * next_return
                advantages = returns - values

        # flatten the batch
        b_obs = obs.reshape((-1,) + (state_dim,))
        b_logprobs = logprobs.reshape(-1)
        b_actions = actions.reshape((-1,) + (len(action_dims),)) #Need to fix dims later
        b_advantages = advantages.reshape(-1)
        b_returns = returns.reshape(-1)
        b_values = values.reshape(-1)

        # Optimizing the policy and value network
        b_inds = np.arange(batch_size)
        clipfracs = []
        for epoch in range(update_epochs):
            np.random.shuffle(b_inds)
            for start in range(0, batch_size, minibatch_size):
                end = start + minibatch_size
                mb_inds = b_inds[start:end]

                _, newlogprob, entropy, newvalue = agent.get_action_and_value(b_obs[mb_inds], b_actions.long()[mb_inds])
                logratio = newlogprob - b_logprobs[mb_inds]
                ratio = logratio.exp()

                with torch.no_grad():
                    # calculate approx_kl http://joschu.net/blog/kl-approx.html
                    old_approx_kl = (-logratio).mean()
                    approx_kl = ((ratio - 1) - logratio).mean()
                    clipfracs += [((ratio - 1.0).abs() > clip_coef).float().mean().item()]

                mb_advantages = b_advantages[mb_inds]
                if norm_adv:
                    mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

                # Policy loss
                pg_loss1 = -mb_advantages * ratio
                pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - clip_coef, 1 + clip_coef)
                pg_loss = torch.max(pg_loss1, pg_loss2).mean()

                # Value loss
                newvalue = newvalue.view(-1)
                if clip_vloss:
                    v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                    v_clipped = b_values[mb_inds] + torch.clamp(
                        newvalue - b_values[mb_inds],
                        -clip_coef,
                        clip_coef,
                    )
                    v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                    v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                    v_loss = 0.5 * v_loss_max.mean()
                else:
                    v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()

                entropy_loss = entropy.mean()
                loss = pg_loss - ent_coef * entropy_loss + v_loss * vf_coef

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
                optimizer.step()

            if target_kl is not None:
                if approx_kl > target_kl:
                    break

        y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
        var_y = np.var(y_true)
        explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

        # # TRY NOT TO MODIFY: record rewards for plotting purposes
        # writer.add_scalar("charts/learning_rate", optimizer.param_groups[0]["lr"], global_step)
        # writer.add_scalar("losses/value_loss", v_loss.item(), global_step)
        # writer.add_scalar("losses/policy_loss", pg_loss.item(), global_step)
        # writer.add_scalar("losses/entropy", entropy_loss.item(), global_step)
        # writer.add_scalar("losses/old_approx_kl", old_approx_kl.item(), global_step)
        # writer.add_scalar("losses/approx_kl", approx_kl.item(), global_step)
        # writer.add_scalar("losses/clipfrac", np.mean(clipfracs), global_step)
        # writer.add_scalar("losses/explained_variance", explained_var, global_step)
        # print("SPS:", int(global_step / (time.time() - start_time)))
        # print(f"Max reward: {np.max(rewards.cpu().numpy())}")
        print(f"Total reward: {np.sum(rewards.cpu().numpy())}")
        # writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)


Total reward: 12.0
Total reward: 19.0
Total reward: 17.0
Total reward: 30.0
Total reward: 15.0
Total reward: 27.0
Total reward: 47.0
Total reward: 28.0
Total reward: 23.0
Total reward: 32.0
Total reward: 15.0
Total reward: 19.0
Total reward: 16.0
Total reward: 27.0
Total reward: 16.0
Total reward: 21.0
Total reward: 34.0
Total reward: 22.0
Total reward: 23.0
Total reward: 39.0
Total reward: 29.0
Total reward: 45.0
Total reward: 24.0
Total reward: 20.0
Total reward: 14.0
Total reward: 41.0
Total reward: 20.0
Total reward: 17.0
Total reward: 14.0
Total reward: 10.0
Total reward: 27.0
Total reward: 49.0
Total reward: 26.0
Total reward: 63.0
Total reward: 24.0
Total reward: 16.0
Total reward: 29.0
Total reward: 14.0
Total reward: 16.0
Total reward: 14.0
Total reward: 20.0
Total reward: 55.0
Total reward: 52.0
Total reward: 19.0
Total reward: 30.0
Total reward: 34.0
Total reward: 27.0
Total reward: 20.0
Total reward: 20.0
Total reward: 9.0
Total reward: 23.0
Total reward: 27.0
Total reward:

KeyboardInterrupt: 

In [ ]:
env.step(0)

c:\ProgramData\Anaconda3\envs\tf38\lib\site-packages\gymnasium\envs\classic_control\cartpole.py:180: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


(array([-0.09391186, -1.1373829 ,  0.2711489 ,  2.1417098 ], dtype=float32),
 0.0,
 True,
 False,
 {})